Data Preprocessing and Loading

In [1]:
from src.data.data import DataProcessor, TimeSeriesDataset, TimeSeriesDataLoader

# Preprocess data
data_processor = DataProcessor(filename='bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv', 
                               start='2021-03-30 00:00:00', 
                               end='2021-03-31 00:00:00', 
                               verbose=True)
data_processor.prepare_data(exclude_columns_detrend=['SMA', 'RSI', 'MACD', 'log_ret', 'percent_ret'],
                            exclude_columns_scale=['log_ret', 'percent_ret'])

# Create dataloaders
train_loader, val_loader, test_loader = data_processor.create_dataloaders(window=10, 
                                                                          batch_size=64,
                                                                          exclude_input_columns=[''])

data_processor.train.head()

Data loaded --------- data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv
Using data from 2021-03-30 00:00:00 to 2021-03-31 00:00:00 with 1440 rows and 12 columns


,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,SMA,RSI,MACD,log_ret,percent_ret
Timestamp,,,,,,,,,,,,
2021-03-30 00:00:00,0.258497,0.266949,0.253401,0.273153,0.061575,0.060983,0.257939,0.254462,0.520251,0.374126,0.001223,0.012236
2021-03-30 00:01:00,0.240734,0.229228,0.201345,0.192764,0.014084,0.013929,0.221845,0.253190,0.235106,0.320573,-0.003268,-0.032627
2021-03-30 00:02:00,0.192115,0.241298,0.195447,0.246887,0.056052,0.055489,0.247135,0.253666,0.419462,0.316518,0.002201,0.022038
2021-03-30 00:03:00,0.252264,0.251316,0.242924,0.247070,0.007474,0.007397,0.239168,0.252559,0.420015,0.313214,0.000007,0.000075
2021-03-30 00:04:00,0.245555,0.248202,0.228727,0.254376,0.007140,0.007062,0.224739,0.252938,0.443036,0.315564,0.000297,0.002968
